In [51]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import pyarrow.parquet as pq
import pyarrow as pa
import os

In [52]:
chile_df = pd.read_csv('data/chile.csv',  low_memory=False)
argentina_df = pd.read_csv('data/argentina.csv',  low_memory=False)
antartica_df = pd.read_csv('data/antartica.csv',  low_memory=False)

In [53]:
#info que muestre todos los datos 
chile_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9481 entries, 0 to 9480
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no  

In [54]:
argentina_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31881 entries, 0 to 31880
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no

In [55]:
antartica_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11311 entries, 0 to 11310
Data columns (total 146 columns):
 #    Column               Dtype  
---   ------               -----  
 0    occurrence_no        int64  
 1    record_type          object 
 2    reid_no              float64
 3    flags                object 
 4    collection_no        int64  
 5    accepted_name        object 
 6    accepted_attr        float64
 7    accepted_rank        object 
 8    accepted_no          float64
 9    early_interval       object 
 10   late_interval        object 
 11   max_ma               float64
 12   min_ma               float64
 13   ref_author           object 
 14   ref_pubyr            int64  
 15   reference_no         int64  
 16   phylum               object 
 17   phylum_no            object 
 18   class                object 
 19   class_no             object 
 20   order                object 
 21   order_no             object 
 22   family               object 
 23   family_no

In [56]:
pd.set_option('display.max_columns', None)
chile_df.head()

,occurrence_no,record_type,reid_no,flags,collection_no,accepted_name,accepted_attr,accepted_rank,accepted_no,early_interval,late_interval,max_ma,min_ma,ref_author,ref_pubyr,reference_no,phylum,phylum_no,class,class_no,order,order_no,family,family_no,genus,genus_no,subgenus_no,primary_name,primary_reso,subgenus_name,subgenus_reso,species_name,species_reso,image_no,plant_organ,plant_organ2,abund_value,abund_unit,taxon_environment,environment_basis,motility,motility_basis,life_habit,life_habit_basis,vision,vision_basis,diet,diet_basis,reproduction,reproduction_basis,ontogeny,ontogeny_basis,ecospace_comments,composition,architecture,thickness,reinforcement,taphonomy_basis,lng,lat,cc,state,county,latlng_basis,latlng_precision,geogscale,geogcomments,paleomodel,geoplate,paleoage,paleolng,paleolat,paleomodel2,geoplate2,paleoage2,paleolng2,paleolat2,paleomodel3,geoplate3,paleoage3,paleolng3,paleolat3,formation,stratgroup,member,formation.1,stratgroup.1,member.1,stratscale,zone,localsection,localbed,localbedunit,localorder,regionalsection,regionalbed,regionalbedunit,regionalorder,stratcomments,lithdescript,lithology1,lithadj1,lithification1,minor_lithology1,fossilsfrom1,lithology2,lithadj2,lithification2,minor_lithology2,fossilsfrom2,lithdescript.1,lithology1.1,lithadj1.1,lithification1.1,minor_lithology1.1,fossilsfrom1.1,lithology2.1,lithadj2.1,lithification2.1,minor_lithology2.1,fossilsfrom2.1,environment,tectonic_setting,geology_comments,time_bins,time_contain,time_major,time_buffer,time_overlap,collection_type,collection_methods,museum,collection_coverage,collection_size,rock_censused,collectors,collection_dates,collection_comments,taxonomy_comments,primary_reference,authorizer_no,enterer_no,modifier_no,authorizer,enterer,modifier
0,16417,occ,NaN,NaN,1314,Leionucula,NaN,genus,16064.0,Maastrichtian,NaN,72.1,66.0,Jablonski and Raup,1999,40,Mollusca,7805,Bivalvia,16005,Nuculida,16006,Nuculidae,61454,Leionucula,16064,NaN,Leionucula,NaN,NaN,NaN,ceciliana,NaN,347.0,NaN,NaN,NaN,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculidae,facultatively mobile,Nuculidae,infaunal,Nuculidae,blind,Bivalvia,"deposit feeder, suspension feeder",Nuculidae,NaN,NaN,accretion,Nuculidae,NaN,aragonite,NaN,thin,minor ribbing,Nuculidae,-73.0,-37.0,CL,NaN,NaN,NaN,degrees,basin,Chile (Central),gplates,291,mid,-58.83,-36.23,scotese,291,mid,-61.11,-38.86,seton,291,mid,-56.61,-39.51,NaN,NaN,NaN,NaN,NaN,NaN,formation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maastrichtian,Maastrichtian,Maastrichtian,Maastrichtian,Maastrichtian,NaN,NaN,NaN,some macrofossils,NaN,NaN,NaN,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
1,16418,occ,NaN,NaN,1314,Nuculana,NaN,genus,16083.0,Maastrichtian,NaN,72.1,66.0,Jablonski and Raup,1999,40,Mollusca,7805,Bivalvia,16005,Nuculanida,97194,Nuculanidae,61453,Nuculana,16083,NaN,Nuculana,NaN,NaN,NaN,amuriensis,NaN,347.0,NaN,NaN,NaN,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculanidae,facultatively mobile,Nuculanidae,infaunal,Nuculanidae,blind,Bivalvia,"deposit feeder, suspension feeder",Nuculanidae,NaN,NaN,accretion,Nuculanidae,NaN,aragonite,NaN,thin,minor ribbing,Nuculanidae,-73.0,-37.0,CL,NaN,NaN,NaN,degrees,basin,Chile (Central),gplates,291,mid,-58.83,-36.23,scotese,291,mid,-61.11,-38.86,seton,291,mid,-56.61,-39.51,NaN,NaN,NaN,NaN,NaN,NaN,formation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maastrichtian,Maastrichtian,Maastrichtian,Maastrichtian,Maastrichtian,NaN,NaN,NaN,some macrofossils,NaN,NaN,NaN,NaN,NaN,"history: new-508, DJ 1992, replaces old m-24 (...",D. Jablonski and D. M. Raup. 1999. Unpublished...,29,29,4,D. Jablonski,D. Jablonski,J. Alroy
2,16419,occ,NaN,NaN,1314,Nuculana,NaN,genus,16083.0,Maastrichtian,NaN,72.1,66.0,Jablonski an

Como grupo fuimos viendo manualmente que columnas nos servirian para poder responder las preguntas planteadas en la propuesta inicial, y llegamos a un concenso de elegir las siguientes 19 columnas como base para poder responder estas.

In [70]:
columns = [
    'species_name',
    'occurrence_no',
    'accepted_name',
    'accepted_rank',
    'early_interval',
    'late_interval',
    'max_ma',
    'min_ma',
    'phylum',
    'class',
    'order',
    'family',
    'genus',
    'lat',
    'lng',
    'diet',
    'geogscale',
    'life_habit',
    "environment",
    "taxon_environment",
    "primary_name"
]

#Hacer que los df solo tengan las columnas que necesitamos
chile_df = chile_df[columns].copy()
argentina_df = argentina_df[columns].copy()
antartica_df = antartica_df[columns].copy()

In [58]:
chile_df

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,regionalsection,life_habit,environment,taxon_environment,primary_name
0,ceciliana,16417,Leionucula,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculida,Nuculidae,Leionucula,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,NaN,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Leionucula
1,amuriensis,16418,Nuculana,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Nuculanidae,Nuculana,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,NaN,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculana
2,cuneiformis,16419,Nuculana,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Nuculanidae,Nuculana,-37.000000,-73.000000,"deposit feeder, suspension feeder",basin,NaN,infaunal,NaN,"coastal,inner shelf,outer shelf,oceanic",Nuculana
3,levitestata,16420,Yoldia,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculanida,Yoldiidae,Yoldia,-37.000000,-73.000000,deposit feeder,basin,NaN,infaunal,NaN,"inner shelf,outer shelf,oceanic",Yoldia
4,pencana,16421,Neiloides,genus,Maastrichtian,NaN,72.10,66.000,Mollusca,Bivalvia,Nuculida,NO_FAMILY_SPECIFIED,Neiloides,-37.000000,-73.000000,deposit feeder,basin,NaN,infaunal,NaN,NaN,Neiloides
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9476,sp.,1659402,Ostrea,genus,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Ostreida,Ostreidae,Ostrea,-50.884998,-72.625000,suspension feeder,NaN,NaN,epifaunal,basinal (siliciclastic),"hypersaline,marine,brackish",Ostrea
9477,antarctica,1659403,Cucullaea antarctica,species,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Arcida,Cucullaeidae,Cucullaea,-50.884998,-72.625000,suspension feeder,NaN,NaN,infaunal,basinal (siliciclastic),NaN,Cucullaea
9478,sp.,1659404,Nucula,genus,Turonian,Campanian,93.90,72.100,Mollusca,Bivalvia,Nuculida,Nuculidae,Nucula,-50.884998,-72.625000,"deposit feeder, suspension feeder",NaN,NaN,infaunal,basinal (siliciclastic),"coastal,inner shelf,outer shelf,oceanic",Nucula
9479,saladensis,1660104,Micrauchenia saladensis,species,Tortonian,Messinian,11.63,5.333,Chordata,Mammalia,Panameriungulata,Macraucheniidae,Micrauchenia,-27.684723,-70.949997,herbivore,NaN,NaN,scansorial,coastal indet.,terrestrial,Micrauchenia


In [60]:
obtener_duplicados_chile = chile_df.duplicated(keep=False)
datos_duplicados_chile = chile_df[obtener_duplicados_chile]
datos_duplicados_chile

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,regionalsection,life_habit,environment,taxon_environment,primary_name


In [61]:
obtener_duplicados_argentina = chile_df.duplicated(keep=False)
datos_duplicados_argentina = chile_df[obtener_duplicados_argentina]
datos_duplicados_argentina

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,regionalsection,life_habit,environment,taxon_environment,primary_name


In [69]:
obtener_duplicados_antartica = antartica_df.duplicated(keep=False)
datos_duplicados_antartica = antartica_df[obtener_duplicados_antartica]
datos_duplicados_antartica

,species_name,occurrence_no,accepted_name,accepted_rank,early_interval,late_interval,max_ma,min_ma,phylum,class,order,family,genus,lat,lng,diet,geogscale,regionalsection,life_habit,environment,taxon_environment,primary_name


In [63]:
chile_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9481 entries, 0 to 9480
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species_name       9481 non-null   object 
 1   occurrence_no      9481 non-null   int64  
 2   accepted_name      9343 non-null   object 
 3   accepted_rank      9343 non-null   object 
 4   early_interval     9481 non-null   object 
 5   late_interval      1605 non-null   object 
 6   max_ma             9481 non-null   float64
 7   min_ma             9481 non-null   float64
 8   phylum             9337 non-null   object 
 9   class              9292 non-null   object 
 10  order              9123 non-null   object 
 11  family             8914 non-null   object 
 12  genus              8574 non-null   object 
 13  lat                9481 non-null   float64
 14  lng                9481 non-null   float64
 15  diet               8749 non-null   object 
 16  geogscale          6426 